# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='../chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    current_height = browser.execute_script("return document.body.scrollHeight;")
    
    if current_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = current_height
        continue

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 媽媽請領育嬰留停津貼　爸爸可接棒再請領...父母合計最長可請領1年

▲勞保局表示，同1名子女父母分別可以請領，合計最長時間為12個月。（圖／取自免費圖庫Pixabay）
記者余弦妙／台北報導
台灣面臨少子化問題，因此政府對於生育福利措施也更加重視，因此不少民眾也會逐漸注意到自己的福利，像是育嬰留停津貼該如何請領相關問題，對此，勞保局表示，若父母均為就業保險被保險人並符合請領條件者，可分別申請6個月津貼，但期間不得重疊，同1名子女的父母，可分別請領，合計最長可請領12個月。
不過，如果生育的如果是雙胞胎或多胞胎，且未滿3歲子女，依性別工作平等法規定同時辦理育嬰留職停薪，且符合育嬰留職停薪津貼的請領條件者，得同時請領不同子女的育嬰留職停薪津貼。
總而言之，同1名子女的父母分別可以請領，合計最長時間為12個月，若是生了二胎、三胎甚至更多，就應該先後個別提出申請。
----------------------------------------------------------------------

[1] 見證府城時代劇！　台南中西區「阿公級地標」88歲了 !

記者洪正達／高雄報導
台南林百貨是國內外相當知名的地標，2019年12月已邁入了第88個年頭，見證了日治、國民政府遷台、經濟奇蹟等時代，原本處於荒廢的林百貨在台南市政府前幾年的推動、整修下，已經以最初的樣貌呈現在國人面前，目前市府也正籌劃各類活動為這個88歲的「阿公級」地標慶生。
文化局表示，2019年12月是林百貨88歲生日，台南市政府以「樂來樂愛林」為主題辦理慶生活動，登場府城摩登大遊行，共有25個團體、近500名民眾，日本本庄市吉田信解市長及田中輝好議長也一同參與遊行，從原中國城出發，以1930年代台灣的流行歌為題，沿途透過音樂舞蹈表演重現經典，向日治時期林百貨年代的音樂人物致敬。
府城摩登大遊行舉辦以來，參與人數逐年增加，已成為台南重要的年度藝文盛事，成功建立了一個新台南傳統節慶，更是台南的時代祭。市長黃偉哲表示，台南市政府希望林百貨不僅是台南的地標，更能讓台南的文化歷史深植人心，期待在台南逐漸邁向2024城市歷史400年之際，林百貨的府城摩登大遊

▲大樂透初一報喜！2.4億頭獎一注獨得，獎落台東「鑫富發彩券行」。（圖／翻攝自GoogleMaps）
記者柯沛辰／台北報導
大樂透2.4億頭獎25日一注獨得，獎落台東「鑫富發彩券行」。春節大紅包9中6部分，今晚一共開出37組，總中獎注數46注，其中31組獨得100萬元獎金，4組兩注均分各得50萬元，1組三注均分、1組四注均分。明晚初二26日將繼續加開9個獎號，對中任6個號碼就有機會抱走百萬獎金。
▲大樂透一注獨得，獎落台東。（圖／台彩，下同）
台彩加碼春節大紅包，從1月23日小年夜起至2月2日， 大樂透連續11天開獎，每次加開9個號碼，只要對中6個號碼，即有機會獲得100萬元紅包，活動期間有360組百萬大紅包，直到2月7日或提前送完為止。
目前已送出91組大紅包，如果1月27日360組百萬大紅包未全數送出，1月28日起將繼續加開9個獎號，直到360組獎金全數送出或2月7日活動截止。
▼春節紅包對中組數。
▼春節紅包開出商店。
▲台彩春節加碼行事曆。（圖／台彩）
----------------------------------------------------------------------

[12] 每坪破百萬！　中西區新光三越中山店三度拿下「台南地王」

記者洪正達／台南報導
台南市中西區新光三越中山店再度以每坪104.8萬元的價格第三次蟬聯台南地王，新光三越中山店自從2017年以100萬1657元，小贏1611元的價格擠下舊地王「萬國通路皮件」的100萬44元，至今每年仍維持微幅漲勢，像今年漲幅就達到了1.60％，台南市地政局進一步表示，公告土地現值及公告地價將於2020年1月1日公告。
地政局表示，台南市2020年公告土地現值及公告地價經地價評議委員會評定通過後，公告土地現值及公告地價平均皆微調0.98%，雖然住宅價格指數顯示，台南市由2018年第1季99.63上漲到2019年第2季的104.67，呈現連續5季走揚，累積漲幅達到5.04％，不過在市府「減稅輕賦」的施政方針下，在衡酌公告現值變動、社會經濟狀況、民眾賦稅能力及本市財政需要等因素後，2020年公告地價及公告現值僅微調0.98%，以減輕民眾負擔。而位處中山路、民族路及西華南街路口的地王新光三越中山店，公告土地現值調升為每平方公尺31萬7千元（每坪約為104.8萬元），上漲1.60%，位於北

▲劉青山用青山蘭花萃取花瓣做成保養品走向國際舞台。（圖／工研院提供）
記者姚惠茹／台北報導
保養品拿來做為拜年伴手禮，再適合不過！有業者推出用蘭花天然萃取物做成保養品，不只運用台日專業團隊優化萃取技術，更深受日本人喜愛。
----------------------------------------------------------------------

[23] 台商吹起「鮭魚返鄉風」　大台南地區總投資額破千億大關

記者洪正達／台南報導
台商回流投資台南金額突破1000億元，日前投資台灣事務所審查通過群創光電回台投資案，其中投資台南金額為537.12億元，可創造1130個就業機會。根據統計資料顯示，截至目前為止，投資台南的台商回流投資計有24案，總投資金額將達1111.31億元、創造9579個就業機會，另審查中或洽談中還有2案。
台南市長黃偉哲表示，市政推動產經重鎮主軸，透過環境整備作為、產業措施及積極開發投資標的，吸引投資創造就業。未來無論是回流台商或是投資本市企業，市府推動簡政便民措施，將就廠商設廠所涉及的各項行政流程，採「優先排審」及「以抽換補正代替退件重審」等方式，加速行政審查程序。
▲台商資金大舉回流，台南正積極推動產業聚落成立，圖為永康工業區服務中心。（圖／記者洪正達翻攝）
群創光電的生產基地及銷售網絡遍佈全球，主要生產各式液晶面板及觸控模組，例如：電視用面板、筆記型電腦用面板、車用面板等，供應全球尖端資訊與消費電子客戶，為世界TFT-LCD（薄膜電晶體液晶顯示器）領導廠商。本次受中美貿易戰波及，於臺南科學園區擴建及新建廠房以增加產能。
經發局郭阿梅局長也指出，為提供更優質的服務，新吉工業區服務中心已落成啟用，另今年於9月公告受理第五、六區申請設定地上權及申請租用土地，這次出租價格公告受理申請自即日起至2020年6月30日止，歡迎廠商投資台南。
----------------------------------------------------------------------

[24] 青、壯年理財規劃大不同　爽存退休金先掌握兩投資要點

▲ 青、壯年人都要對老年生活有所準備。（圖／pixabay）
記者李瑞瑾／綜合報導
勞保不確定性、長壽趨勢、老年醫療費用、通膨加乘等因素，影響退休規劃的風險相當多，投信業者表示，在規劃退休理財上，

▲亞馬智慧音箱Amazon Echo。（圖／達志影像／美聯社）
記者陳心怡／外電報導
亞馬遜近年不斷攻入智慧居家服務市場，而亞馬遜推出的Echo智慧音箱也成功入榜。亞馬遜的Echo智慧音箱和Alexa語音助理是最受歡迎的產品之一，目前已售出了超過1億台支援Alexa的設備。智慧音箱的功能也越來越多，現在不僅能播音樂、回答問題，還有許多特別功能，可以幫助使用者更輕鬆使用Amazon Echo。
----------------------------------------------------------------------

[33] 高雄低價購小宅！單層戶數鄰居多　房產業者透露：5年內見真章

▲高雄房地產趨向小宅化。（示意圖／取自免費圖庫Pexels）
記者許宥孺／高雄報導
近年來高雄房地產市場逐漸呈現「小宅化」，建商大多推出低價、小坪數建案，成功吸引首購族目光，不過房仲業者卻指出，小宅單層戶數多，出入人口混雜管理不易，加上方位、採光也是大問題，五年後可以試問購買小宅的消費者就能知道住屋心得。
資深房仲業者指出，買房盡量挑選「三房」較恰當，但是受到「剛性需求」影響，許多消費者沒有餘裕買三房，但首購族又想買新房，就可能退而其次購買兩房小宅，也因此，現在有許多建商都是規劃2房，價格約500萬至700萬，坪數約25坪上下的小宅。
不過業者卻透露，雖然房屋「總價」配合消費者的剛性需求，但是「單價」幾乎都被拉高了，比如說原本一坪20萬，現在幾乎都拉高到21、22萬。
▲房仲業者指出，住戶多就會延伸出電梯塞車問題。（圖／pixabay）
業者指出，小宅低總價就會反映在「小坪數」，規劃單層戶數多就可能延伸許多問題，像是公共設施壽命短、電梯塞車、出入人口混雜管理不易、住戶買賣頻繁造成永遠都有施工期的狀況，另外還有採光、方位問題，建議民眾購屋前要多方考量，也可詢問購買小宅的消費者「住屋心得」，小宅的優點還是缺點，五年內見真章。
►他奶奶的美麗小秘訣
----------------------------------------------------------------------

[34] 別再用「123456」當密碼啦！BBC教你自創好記又不易攻破的安全密碼

▲ 若想保護好自己線上世界，設定一組強大的密碼是必不可少的。（圖／取自免費圖庫Pixabay）
記者

▲ 年節前後刮刮樂銷售總是特別好。（示意圖／記者李毓康攝）
記者李瑞瑾／綜合報導
金孫加持中大獎！新竹縣一名小孩原本不會吵著要買東西，但某天一直吵著要買「有車子的刮刮樂」，讓差點與財神錯過的阿公照著金孫意思，用孫子的生日去買了張「2,000萬超級紅包」刮刮樂，結果順利刮中大獎100萬，讓阿公阿嬤直呼孫子真的是好「金」孫！
新竹縣彩券行店員描述，19日中午一個阿嬤帶著遠從宜蘭回來的女兒及孫子來買彩券，當下孫子一直吵著要買有車子的2,000元刮刮樂，但是阿嬤因為價格考量當下並沒有順著孫子的意願，就這樣離開了彩券行，孫子也只能很難過的離開。
但是晚上在回宜蘭的路上，孫子一直在車上吵著要買有賓士車的刮刮樂，孫子的媽媽此時瞬間閃過一個念頭，孫子明明平常都不會吵著要買東西，但是今天特別的執著，便打了電話回家裡告知阿公阿嬤，阿公一聽，二話不說馬上前往彩券行，用孫子的生日號碼「17號」買了一張「2,000萬超級紅包」帶回家刮，沒想到真的刮中大獎100萬，阿公阿嬤馬上打電話給女兒，直呼真的是我的好「金」孫啊！
----------------------------------------------------------------------

[44] 金孫從新竹吵到宜蘭「買有車車的刮刮樂」　阿公「一個轉念」爽中100萬

▲台彩公司今年推出的2000萬超級紅包刮刮樂。（圖／記者李毓康攝）
記者葉國吏／綜合報導
每年農曆春節台彩公司都會推出大面額的刮刮樂衝買氣，今年推出的2000萬超級紅包除了有頭獎2000萬外，還有加碼賓士車，吸引不少人購買。由於刮刮樂上印有賓士車的模樣，意外吸引小朋友目光，新竹一位小朋友看到車子後吵著要買，疼孫的阿公買了，結果刮出100萬。
新竹一位阿嬤19日中午帶著從宜蘭回娘家的外孫買彩券，當時孫子看到2000元的刮刮樂上印有車子，一直吵著要買，但當時阿嬤覺得一張2000元實在太貴，所以沒有答應就離開。小孩由於沒有買到有車車的刮刮樂，在回宜蘭的路途上一直吵，小孩媽媽覺得不太尋常，突然閃過一個念頭。
媽媽覺得，小孩平常不會吵著要買東西，但今天卻異常的堅持要買有車車的刮刮樂，於是打電話給住在新竹的阿公，阿公聽完後馬上衝到彩券行，並以金孫的的生日號碼「17號」買了一張帶回家刮，結果真的刮到100萬元，讓阿公、阿嬤直呼真的是金孫。
---------------

記者王致凱／台北報導
台彩春節大樂透春節加碼，台彩24日除夕晚間開出109000008期大樂透，頭獎上看1.7億元，獎號分別為21、18、07、44、13、38，特別號03。
除夕春節大紅包加開獎碼為26、32、09、42、35、33、30、03、07。
台彩「春節大紅包」加碼，自23日小年夜起至2月2日，大樂透連續11天開獎，每次加開9個號碼，只要對中6個號碼，即有機會獲得100萬元紅包，活動期間有360組百萬大紅包，直到2月7日或提前送完為止。上述獎號資訊以台彩公布為準。
►「秒殺小魚乾起司條」～最狂春節伴手禮來了！
----------------------------------------------------------------------

[56] 圍爐大餐甩肥油大作戰　Apple Watch陪你一起動起來

▲過年大餐後想來一場「甩肥油大作戰」，那就快戴上Apple Watch做運動。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
過年吃完圍爐大餐後，一家人可能坐在客廳嗑瓜子、吃零食話家常，但體重似乎也悄悄地增加，不過沒關係，如果你手上有配戴Apple Watch，不妨可以上個瑜伽課，並試試體能訓練功能中的「瑜伽」活動類型（須更新至watchOS 5後的版本），來追蹤消耗的動態卡路里以及運動分鐘數吧！
不過，要如何開啟瑜伽活動類型？首先先點開Apple Watch的體能訓練，也就是下圖中間，有綠色圈圈的跑步小人，再上下滑動尋找「瑜伽」，輕點後倒數3秒，即可開始瑜伽活動。
▼點選手錶中的小綠人，往下滑就可找到「瑜珈」模式。（圖／記者邱倢芯攝）
根據官方指出，Apple Watch的瑜伽體能訓練活動類型，能追蹤消耗的動態卡路里、運動分鐘數，以及運動時的心率狀況等。不過Apple Watch並不能提供做瑜伽時的動作指導，建議還是尋找專業的瑜伽教室與老師，再配合Apple Watch搭配使用會比較好。
在瑜珈課結束後，要先喚醒手錶並向左滑動，此時你會看到畫面上有鎖定、新增、結束、繼續4個選項，點選「結束」後，畫面會再跳出此次的運動摘要。
而在瑜伽課程結束後，老師通常會讓你進行「大休息」，藉此調整你的呼吸，平緩心跳速度等。這時候也可以配合手錶當中的「呼吸」功能，來調整呼吸。
若要使用呼吸功能，可以在功能頁面中找到一個藍色花狀的圖案，開啟功能後的畫面如下圖

▲立蕓國際生醫集團總經理余朱青。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
農曆春節就是大吃美食的節日，如何吃得輕巧又健康？營養師出身的立蕓國際生醫集團總經理余朱青分享自己在過年時的飲食原則，除了在圍爐火鍋、零食餅乾、長輩菜單等方面需要注意的地方以外，還有如何在大吃大喝以後，也能維持身材的小方法。
首先，余朱青分享，春節過年的時候，因為連假放鬆的關係，生活和飲食都會和平常不一樣，尤其是年節大餐，往往一個連假下讓人增體不少。余朱青遵循幾個原則來讓自己吃得營養又健康，即使吃大魚大肉，每天也一定要吃三個拳頭份量的蔬菜水果來維持營養均衡。
談到除夕不可少的圍爐火鍋，余朱青說，自己會少吃加工食品，而且容易吸油的蔬菜也會把油去掉再吃，還有如果一定要搭配醬汁的話，因為傳統醬汁的鈉含量比較高，所以自己會選擇比較清淡的日式醬油，並用少許檸檬汁來調味。
▲立蕓國際生醫集團總經理余朱青為旗下朵薇診所營養師。（圖／記者姚惠茹攝）
至於春節連假必備的零食，可分為瓜子、花生、杏仁、核桃等堅果類，以及魷魚絲、肉乾等蛋白質類，還有糖果、餅乾、薯片等碳水化合物等，余朱青建議，三大類中可以先吃堅果類，而且份量要控制在一個拳頭內，再吃蛋白質來增加飽足感，最後吃碳水化合物就會相對吃得比較少。
針對長輩在春節的飲食菜單，余朱青表示，主要應該避免高膽固醇、高鹽、高油等食物，因為年長者的代謝不好，再來則是要避免不好消化的食物，像是年糕、糯米類等類別，所以比較建議選擇以蒸煮、清炒的方式烹調的食物較為健康。
最後，余朱青強調，如果真的不可免俗的大吃大喝，還是有小方法可以讓身體快速減輕負擔，那就是要喝足水分，因為喝水可以讓身體代謝，並且讓腸胃消化維持正常運作，或是可以搭配酵素、益生菌來幫助代謝，最重要的則是保持正常的作息時間，讓過年連假同樣做到健康生活。
----------------------------------------------------------------------

[66] 2020年AR四大趨勢！　硬體製造商減少、軟體更多、無線技術更方便、硬體更好

►AR擴增實境將成為趨勢。（圖／翻攝自 Youtube）
記者謝仁傑／綜合報導
改進的硬體
去年，許多更好的AR硬體零件開始進入市場，其中最令人驚訝的是包括更小且具更高分辨率的顯示器，以及為AR和混合實境設備，訂製設計的新晶片組。今年大

▲刮刮樂又有百萬得主出爐。（資料照／記者紀佳妘攝）
財經中心／台北報導
刮刮樂又出現100萬元大獎幸運得主了！台彩表示，位於北市南海路的「功頂彩券行」店員描述，一位年約40多歲的男性常客，買了三張2,OOO元的刮刮樂「2,000萬超級紅包」，結果刮到第二張就中了百萬大獎。
店家表示，這位常客平常就喜歡購買高面額的刮刮樂，當天特別向他介紹今年2,OOO元刮刮樂「2,000萬超級紅包」，100萬獎項比去年多了1OO個，「200萬加賓士車」獎項也比去年多了2個，過年可可試一試手氣。
該位男子當場買了三張，沒想到刮到第二張時就刮出一百萬元大獎，趕緊請店員幫忙確認是否真的刮出大獎，還直呼心臟一直蹦蹦跳，不敢置信。
投注站店員表示，自從民國107年萬大-中和線捷運工程開挖後，工程圍籬圍在南海路邊，就沒有再開出特別大的獎項，生意也受影響，後來經指點要請武財神關老爺鎮店幫店裡闢出一條財路，所以特地從行天宮請來武財神關老爺到店內。
店員說，民國108年1月初迎神入店後，當年2月14日店裡馬上開出今彩539的頭獎。而今年又有客人幸運刮出一百萬的大獎，店員直呼真是太靈驗了，希望關老爺再發功，讓店裡大獎不斷！
----------------------------------------------------------------------

[76] 年節賺外快勿忘權益　留意雇主未幫你投勞健保

▲節慶時間兼差往往因為時間短，忽略了投保勞保的必要性，即使是短期的兼差打工，權益和正職員工都是一樣的。（圖／記者張一中攝）
記者余弦妙／台北報導
不少民眾都會趁著年節期間打工賺取外快。根據1111人力銀行調查，有意兼差打工的族群當中，有7成7上班族會因為2020年基本工資調升增加兼職意願，另外2成3不受影響，照原計畫打工，顯示基本工資提升，會讓有意兼差的族群在打工意願上更加積極，積少成多增加收入。
而在將近一個月的兼職過程中，上班族期待可以賺進1萬2,028元，等於調查中上班族平均薪資的35%。
社團法人台灣服務業發展協會理事長劉福財表示，年節期間服務業因為延長營業時間，增加對人力的需求，從企業的角度出發，人力成本就會因此提高，如何在當中取得平衡，自我調整、創新改革都是很重要的，透過創新帶出企業的附加價值，才能創造更好的福利給員工。
調查顯示，3成3的兼差上班族在打工時遭遇過不合理對待，

▲各國央行繼續買進黃金、投資人的資金需求等，繼續推升黃金價格，今年有望突破每盎司1600美元。 （圖／達志／示意圖）
記者吳靜君／台北報導
隨著2019年中美貿易戰緊張，黃金價格在去年5月到9月上漲了兩成左右，今年黃金表現如何呢？台銀貴金屬部副理楊天立表示，今年1月中，雖然發生美伊衝突使黃金價格有大幅波動，今年整體而言，央行和投資人需求仍在，使得黃金價格底部墊高，預期黃金價格在每盎司1635到1675美元之間。
即使黃金的價格下跌，楊天立表示，由去（2019）年來看，黃金大約在1450美元有很好的支撐，所以即使今年黃金價格遇到壓力，跌破1450美元機會也不大。
首先要看黃金現貨需求。楊天立表示，由於印度課稅和中國大陸經濟成長趨緩，所以消費性黃金需求可能會被放緩；另外，美國如果經濟維持正向，對黃金消費性、現貨需求則穩定；歐洲對黃金需求也維持平盤。
從整體供需來看，黃金每年大約供給3000多公噸，今年則是繼續維持；整體而言，現貨的供需維持平衡，對於黃金價格影響平衡穩定。
黃金價格增長　各國央行繼續買進
今年影響黃金價格主要是來自資金面，資金面有兩股力量，其一就是中央銀行買盤；另外一股力量就是黃金ETF和期貨買盤。
楊天立表示，各國中央銀行買盤去年前11月已經買進超過651公噸；所以預估今年買進速度雖然可能放緩，但仍有再買進650公噸的潛力。
楊天立表示，中央銀行買黃金的目的是分散風險，因為央行外匯準備龐大，主要資產包含外幣、外幣計價公債，有鑑於全世界經濟金融情勢變化，所以許多新興國家央行曾持黃金準備；另一作用是，萬一金融體系發生風險，或者發生貨幣戰，準備黃金太少，受傷會比較大，所以增持黃金是以保障自己為作用，去年買進黃金比較多的國家包含俄羅斯、中國大陸、土耳其等。
ETF、黃金期貨　投資人投資黃金需求仍然存在
楊天立表示，除了分散風險，投資人對黃金需求仍存在。因黃金已經符合大的投資機構的買進滿足點，所以投資人會積極補進黃金，並且藉由價差大賺黃金財；至於個人部分，2019年的投資需求還是不錯，期貨的未平倉部位還處於蠻高水位、現貨也有還有不錯表現，所以預估今年個人投資人對於黃金投資需求仍然存在。
▲美國經濟也會牽動著今年黃金價格的走勢。（圖／路透、示意圖）
美國經濟下滑、股市下跌　黃金甚至攀1700美元
至於黃金風險部分，楊天立表示，因為中美貿易戰將要簽署第一階段協議，中美

In [6]:
browser.quit();